## Multirun experiment pipeline (WheatFspm)

The following notebook establishes a generalized pipeline for evaluating a computing reservoir against a given task, given multiple experimental runs of the same reservoir.


In [49]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src

In [50]:
DATASET_NEMA_H0 = '../datasets/dataset_NEMA_NEMA_H0.csv'
DATASET_NEMA_H3 = '../datasets/dataset_NEMA_NEMA_H3.csv'
DATASET_NEMA_H15 = '../datasets/dataset_NEMA_NEMA_H15.csv'

### Loading the datasets

These datasets were collected and converted in the WheatFspm repository.

There are three simulations made available in the WheatFspm repository that are useable for RC experiments: NEMA H0, H3 and H15.

We can try using these datasets in two different ways:

1. Treat every dataset as a separate plant, training a readout for each simulation run.
2. Concatenating the three datasets as observed behavior of a single plant.

In [51]:
from src.model.rc_dataset import ExperimentDataset

dataset_nema_h0 = ExperimentDataset(csv_path=DATASET_NEMA_H0)
dataset_nema_h3 = ExperimentDataset(csv_path=DATASET_NEMA_H3)
dataset_nema_h15 = ExperimentDataset(csv_path=DATASET_NEMA_H15)

datasets = [
  ('NEMA_H0', dataset_nema_h0), 
  ('NEMA_H3', dataset_nema_h3), 
  ('NEMA_H15', dataset_nema_h15)
]

### Defining targets and observed state variables

These were selected in a previous notebook, `2022_03_23_wheatfspm_dataset_inspection.ipynb` and are defined in a config file for reuse among notebooks.

In [52]:
%reload_ext autoreload
%autoreload 2 

from model_config import targets, state_variables

print(f'Targets:')
for target in targets:
  print(f'\t- {target}')

print(f'\nState variables:')
for state_var in state_variables:
  print(f'\t- {state_var}')

Targets:
	- input_air_temperature
	- input_humidity
	- input_Wind
	- input_PARi
	- output__axes__Total_Transpiration
	- output__axes__C_exudated
	- output__axes__SAM_temperature
	- output__axes__delta_teq
	- output__axes__sum_respi_shoot
	- output__organ_roots__N_exudation

State variables:
	- state__An
	- state__Transpiration
	- state__S_Sucrose
	- state__Ts
	- state__gs
	- state__Ag
	- state__Tr
	- state__sucrose
	- state__Rd
	- state__sum_respi
	- state__Photosynthesis
	- state__PARa


### Data preprocessing, grouping and train-test splitting

The available datasets will be processed into 4 datasets:

- NEMA_H0
- NEMA_H3
- NEMA_H15
- NEMA_COMBINED (concatenated as data from the same plant)

In [53]:
from src.learning.preprocessing import generate_mask


WARMUP_STEPS = 4 * 24
DAY_MASK = generate_mask(5, 21)

In [63]:
from wheatfspm_pipeline_utils import preprocess_data, group_by_day, train_test_split_alternating
from wheatfspm_pipeline_utils import direct_target_generator, direct_reservoir_generator
from wheatfspm_pipeline_utils import preprocess_raw_X
from model_config import max_time_step

def generate_X_y_groups(datasets, target_generator, state_generator):
  """Generates X, y and groups arrays for each dataset, plus a concatenated dataset.
     NOTE: The groups in the concatenated dataset are such that the same calendar day is in the same group.

     Also generates a baseline dataset where the reservoir is just a combination of all environmental inputs.
  """
  data = {}

  # Preprocess the data for each dataset
  for name, dataset in datasets:
    target_data = next(target_generator(dataset, target, name))
    reservoir_data = next(state_generator(dataset, state_var, name))
    X_raw, y_raw = preprocess_data(dataset, target_data, reservoir_data, WARMUP_STEPS, DAY_MASK)
    X, y = X_raw[0, :, :], y_raw[0, :]
    groups = group_by_day(X, DAY_MASK)
    data[name] = (X, y, groups)

  # Generate the concatenated dataset
  all_arrays = list(data.values())
  X_combined = np.concatenate(list(map(lambda x : x[0], all_arrays)))
  y_combined = np.concatenate(list(map(lambda x : x[1], all_arrays)))
  groups_combined = np.concatenate(list(map(lambda x : x[2], all_arrays)))
  data['combined'] = (X_combined, y_combined, groups_combined)

  # Generate the baseline dataset
  baseline_name, baseline_dataset = datasets[0]
  input_vars = baseline_dataset.get_input_variables()
  X_baseline = np.empty((max_time_step[baseline_name], len(input_vars)))
  for i, var in enumerate(input_vars):
    X_baseline[:, i] = baseline_dataset.get_target(var, baseline_name)[:max_time_step[baseline_name]]
  X_baseline = preprocess_raw_X(X_baseline, WARMUP_STEPS, DAY_MASK)
  y_baseline = data[baseline_name][1]
  groups_baseline = data[baseline_name][2]
  data[f'baseline_{baseline_name}'] = X_baseline, y_baseline, groups_baseline

  return data

In [64]:
TARGET = targets[0]
STATE_VAR = state_variables[0]

preprocessed_data = generate_X_y_groups(datasets, direct_target_generator, direct_reservoir_generator)

for name, (X, y, groups) in preprocessed_data.items():
  print(f'{name}:')
  print(f'\tX: {X.shape}')
  print(f'\ty: {y.shape}')
  print(f'\tgroups: {len(np.unique(groups))} (shape {groups.shape})')

NEMA_H0:
	X: (400, 13)
	y: (400,)
	groups: 25 (shape (400,))
NEMA_H3:
	X: (512, 13)
	y: (512,)
	groups: 32 (shape (512,))
NEMA_H15:
	X: (544, 13)
	y: (544,)
	groups: 34 (shape (544,))
combined:
	X: (1456, 13)
	y: (1456,)
	groups: 34 (shape (1456,))
baseline_NEMA_H0:
	X: (400, 4)
	y: (400,)
	groups: 25 (shape (400,))


### Model definition

- Readout model is a standard RidgeRegression model with intercept term and CV-tuned regularization strength $\alpha$.
- CV search grid is a progression of logarithmicly spaced values for regularization strength $\alpha$.
- CV and testing metric is NMSE.

In [65]:
N_FOLDS = 3;

In [66]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold

from src.learning.scorers import nmse_scorer

# Define model
readout = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])

# define search grid
search_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

# define cross-validation and testing metric
scorer = nmse_scorer

# Define CV fold strategy
folds = GroupKFold(n_splits=N_FOLDS)

### Generating a manifest of all experiments to run

Currently we are only benchmarking direct target prediction, but in the future there will be other tasks generated from the base targets as well. These will be generated in this section.


In [67]:
TARGETS = [(target_name, direct_target_generator) for target_name in targets]
STATE_VARS = [(state_var, direct_reservoir_generator) for state_var in state_variables]

TRAIN_TEST_RATIO = 1

### Fitting all readout functions

Process:

- For each target:
  - For each observed state variable:
    - For each dataset:
      1. Preprocess the data
      2. Fit for each dataset
      3. Store the resulting training, cross-validation and test scores.

In [72]:
from tqdm import tqdm
from src.learning.training import perform_gridsearch


total_loops = len(targets) * len(state_variables) * (len(datasets) + 2)
print(f'Performing {total_loops} fits...')


models = {}
results = []


with tqdm(total=total_loops) as pbar:

    for target_name, target_generator in TARGETS:
        for state_var, state_generator in STATE_VARS:

            # Preprocess data for model fit
            preprocessed_data = generate_X_y_groups(datasets, target_generator, state_generator)

            # For each dataset combination
            for dataset_name, (X, y, groups) in preprocessed_data.items():
                train, test = train_test_split_alternating(X, y, groups, ratio=TRAIN_TEST_RATIO)

                # fit model
                X_train, y_train, groups_train = train
                model, scores = perform_gridsearch(readout, X_train, y_train, groups_train, folds, search_grid, verbose=False)
                (train_mean, train_std), (cv_mean, cv_std) = scores

                # Determine test score
                X_test, y_test, _ = test
                test_score = scorer(model, X_test, y_test)

                # store results
                models[(target_name, state_var, dataset_name)] = model
                results.append({
                    'target': target_name,
                    'state_var': state_var,
                    'dataset': dataset_name,
                    'test_score': test_score,
                    'train_mean': train_mean,
                    'train_std': train_std,
                    'cv_mean': cv_mean,
                    'cv_std': cv_std
                })

                pbar.update(1)                

Performing 600 fits...


100%|██████████| 600/600 [01:02<00:00,  9.61it/s]


### Storing experiment results

Storing all the fit results into a Pandas dataframe, then storing it as a CSV file for further inspection in another notebook.

In [74]:
results_df = pd.DataFrame.from_dict(results)
results_df.set_index(['target', 'state_var', 'dataset'])
results_df.to_csv(f'test_results.csv')
results_df.head(10)

,target,state_var,dataset,test_score,train_mean,train_std,cv_mean,cv_std
0,input_air_temperature,state__An,NEMA_H0,-0.299655,-0.235367,0.036014,-0.413253,0.094021
1,input_air_temperature,state__An,NEMA_H3,-1.070000,-0.266628,0.016394,-0.576058,0.321516
2,input_air_temperature,state__An,NEMA_H15,-0.842455,-0.824210,0.017468,-0.926458,0.061315
3,input_air_temperature,state__An,combined,-0.721588,-0.671309,0.046691,-1.099954,0.447542
4,input_air_temperature,state__An,baseline_NEMA_H0,-0.981863,-0.949312,0.038248,-1.005481,0.047781
5,input_air_temperature,state__Transpiration,NEMA_H0,-0.526377,-0.263443,0.017583,-0.415047,0.028102
6,input_air_temperature,state__Transpiration,NEMA_H3,-0.417018,-0.522495,0.080975,-0.607136,0.174430
7,input_air_temperature,state__Transpiration,NEMA_H15,-0.667238,-0.768667,0.070744,-0.895744,0.096541
8,input_air_temperature,state__Transpiration,combined,-0.555789,-0.542357,0.062261,-1.151606,0.632378
9,input_air_temperature,state__Transpiration,baseline_NEMA_H0,-0.981863,-0.949312,0.038248,-1.005481,0.047781


In [75]:
results_sorted = results_df.sort_values('test_score', ascending=False)
results_sorted.head(10)

,target,state_var,dataset,test_score,train_mean,train_std,cv_mean,cv_std
345,output__axes__C_exudated,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
165,input_Wind,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
405,output__axes__SAM_temperature,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
285,output__axes__Total_Transpiration,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
465,output__axes__delta_teq,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
225,input_PARi,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
525,output__axes__sum_respi_shoot,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
45,input_air_temperature,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
105,input_humidity,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401
585,output__organ_roots__N_exudation,state__sum_respi,NEMA_H0,-0.162659,-0.068266,0.00494,-0.221011,0.078401


In [77]:
results_sorted_top = results_sorted[results_sorted['test_score'] > -0.5]
results_sorted_top.groupby('target').count()

,state_var,dataset,test_score,train_mean,train_std,cv_mean,cv_std
target,,,,,,,
input_PARi,18,18,18,18,18,18,18
input_Wind,18,18,18,18,18,18,18
input_air_temperature,18,18,18,18,18,18,18
input_humidity,18,18,18,18,18,18,18
output__axes__C_exudated,18,18,18,18,18,18,18
output__axes__SAM_temperature,18,18,18,18,18,18,18
output__axes__Total_Transpiration,18,18,18,18,18,18,18
output__axes__delta_teq,18,18,18,18,18,18,18
output__axes__sum_respi_shoot,18,18,18,18,18,18,18
